# Person Re-ID Image Matching

This is for matching two images using an already trained model.

First, import required modules, 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

# Make sure that caffe is on the python path:
caffe_root = '../' 
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import os



Set Caffe to GPU mode, load the net in the test phase for inference, and configure input preprocessing.

In [2]:
caffe.set_device(0)
caffe.set_mode_gpu()
net = caffe.Net(caffe_root +  'examples/_temp/unsup_net_deploy.prototxt',
                caffe_root + 'rank_scripts/models1/_iter_100000.caffemodel',
                caffe.TEST)
net1 = caffe.Net(caffe_root +  'examples/_temp/unsup_net_deploy.prototxt',
                caffe_root + 'rank_scripts/models1/_iter_100000.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'rank_scripts/market_train_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

We'll set a batch of 50 to demonstrate batch processing. (Note that the batch size can also be changed on-the-fly.)

In [3]:
# set net to batch size of 50
net.blobs['data'].reshape(50,3,64,64)
net1.blobs['data'].reshape(50,3,64,64)

Feed in the image (with some preprocessing) and use a forward pass.

In [4]:
net.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image(caffe_root + 'rank_scripts/images_market/0026_c2s1_001626_00.jpg'))
#check_image =  caffe.io.load_image(caffe_root + 'rank_scripts/images_market/0002_c1s2_050821_02.jpg')
out = net.forward()
vector1=out['fc7']
#plt.imshow(check_image)

In [5]:
net1.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image(caffe_root + 'rank_scripts/images_market/0026_c1s1_002301_00.jpg'))
out1 = net1.forward()
vector2=out1['fc7']

In [6]:
# calculating euclidean distance
sum=0
for i in range(50):
    for j in range(1024):
        diff= vector1[i][j]-vector2[i][j]
        diff=diff*diff
        sum=sum+diff
        
print math.sqrt(sum)

0.204652211086
